# Model 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
hitters = pd.read_csv("hitters.csv")
df = hitters.copy()
df = df.dropna()
y = df["Salary"]
x_ = df.drop(["Salary","League","Division","NewLeague"],axis = 1).astype("float64")
dummies = pd.get_dummies(df[["League","Division","NewLeague"]])
dummies = dummies.drop(["League_A","Division_E","NewLeague_A"],axis = 1).astype("float64")
x = pd.concat([x_,dummies],axis = 1)

In [3]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 42)

In [4]:
from sklearn.linear_model import ElasticNet

In [5]:
elasticnet_model = ElasticNet().fit(x_train,y_train)

C:\Users\isaad\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8487984.79936732, tolerance: 3898.6869563806586
  positive)


In [6]:
elasticnet_model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

In [7]:
elasticnet_model.intercept_

-6.4659556021127855

# Tahmin 

In [8]:
y_tahmin = elasticnet_model.predict(x_test)

In [9]:
from sklearn.metrics import mean_squared_error,r2_score

In [10]:
np.sqrt(mean_squared_error(y_test,y_tahmin))

357.16765481812456

In [11]:
r2_score(y_test,y_tahmin)

0.4107022246932688

# Model Tuning 

In [12]:
from sklearn.linear_model import ElasticNetCV

In [19]:
enet_cvmodel = ElasticNetCV(cv = 10,random_state = 0,normalize = True).fit(x_train,y_train)

In [20]:
enet_cvmodel.alpha_

0.03658140728497728

In [21]:
enet_tune = ElasticNet(alpha = enet_cvmodel.alpha_).fit(x_train,y_train)

C:\Users\isaad\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8283826.972405798, tolerance: 3898.6869563806586
  positive)


In [22]:
y_tahmin_tune = enet_tune.predict(x_test)

In [23]:
np.sqrt(mean_squared_error(y_test,y_tahmin_tune))

356.173038901947

In [24]:
r2_score(y_test,y_tahmin_tune)

0.41397972666510696

In [25]:
#ElasticNet modelinde l1_ratio parametresi 0 olduğunda L2 cezalandırması,1 olduğunda ise
#L1 cezalandırılması yapar.Dolayısıyla 0-1 arasında değişimi aslında cezalandırma
#metodlarının göreceli etkilerini ifade eder.Yukarıda iki yöntemin de eşit etkili olmasını
#istediğimiz için ön tanımlı olan 0.5 değerini bıraktık.
#Yani formülde iki alpha olmasına rağmen bizim bir adet alpha değeri belirlememizin nedeni
#l1_ratio değerine göre ve alphaya göre iki katsayı da ağırlıklarına göre belirleniyor.
#Her bir farklı l1_ratio değeri için de farklı alpha değeri olacak ve ceza terimleri farklı şekilde etkilenecektir.